# 03 — Mô hình Phân lớp AQI (Classification Modelling)
Mục tiêu: huấn luyện mô hình phân lớp AQI (6 lớp), chia tập train/test theo thời gian, đánh giá bằng accuracy + macro-F1, vẽ confusion matrix, và lưu metrics cùng mẫu dự đoán.

In [ ]:
# ===== THAM SỐ CẤU HÌNH =====
DATASET_PATH = 'data/processed/dataset_for_clf.parquet'
CUTOFF = '2017-01-01'
METRICS_PATH = 'data/processed/metrics.json'
PRED_SAMPLE_PATH = 'data/processed/predictions_sample.csv'

In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.classification_library import time_split, train_classifier, AQI_CLASSES

PROJECT_ROOT = Path('..').resolve()
dataset_path = (PROJECT_ROOT / DATASET_PATH).resolve()
metrics_path = (PROJECT_ROOT / METRICS_PATH).resolve()
pred_path = (PROJECT_ROOT / PRED_SAMPLE_PATH).resolve()
metrics_path.parent.mkdir(parents=True, exist_ok=True)
pred_path.parent.mkdir(parents=True, exist_ok=True)


In [ ]:
df = pd.read_parquet(dataset_path)
print('Kích thước:', df.shape)
df[['datetime','station','aqi_class']].head()

In [ ]:
# Chia tập train/test theo thời gian (time-based split)
train_df, test_df = time_split(df, cutoff=CUTOFF)
print('Tập huấn luyện:', train_df.shape, '| Tập kiểm tra:', test_df.shape)
print('Khoảng thời gian test:', test_df['datetime'].min(), '->', test_df['datetime'].max())

In [ ]:
# Huấn luyện mô hình phân lớp
out = train_classifier(train_df, test_df, target_col='aqi_class')
metrics = out['metrics']
pred_df = out['pred_df']

print('Độ chính xác (Accuracy):', metrics['accuracy'])
print('Macro F1-Score:', metrics['f1_macro'])
pred_df.head()

In [ ]:
# Vẽ ma trận nhầm lẫn (Confusion Matrix)
cm = np.array(metrics['confusion_matrix'])
labels = metrics['labels']

plt.figure(figsize=(9, 7))
plt.imshow(cm)
plt.title('Ma trận nhầm lẫn (Confusion Matrix)')
plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
plt.yticks(range(len(labels)), labels)
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, str(int(cm[i, j])), ha='center', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Lưu metrics và mẫu dự đoán
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

pred_df.head(5000).to_csv(pred_path, index=False)
print('Đã lưu metrics:', metrics_path)
print('Đã lưu mẫu dự đoán:', pred_path)

In [ ]:
# Hiển thị báo cáo phân lớp (tóm tắt)
report = metrics['report']
summary = {k: report[k] for k in ['accuracy', 'macro avg', 'weighted avg'] if k in report}
print('Tóm tắt kết quả phân lớp:')
summary